In [ ]:
# !pip install bert_score

In [ ]:
import os
import logging
import transformers
import bert_score
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from bert_score import score, BERTScorer
# from torchmetrics.functional.text.bert import bert_score
from datasets import load_dataset
import matplotlib.pyplot as plt
from matplotlib import rcParams

transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [ ]:
%matplotlib inline
bert_score.__version__

In [ ]:
rcParams["xtick.major.size"] = 0
rcParams["xtick.minor.size"] = 0
rcParams["ytick.major.size"] = 0
rcParams["ytick.minor.size"] = 0

rcParams["axes.labelsize"] = "large"
rcParams["axes.axisbelow"] = True
rcParams["axes.grid"] = True

In [ ]:
project_root = '..'
# dataset_name = 'vblagoje/lfqa_support_docs'
# dir_name = 'lfqa'
dir_name = 'eval'
dataset_name = 'mjphayes/machine_learning_questions'
# dataset_name = 'wikitext'
# dataset_variant = 'wikitext-2-raw-v1'
# dir_name = 'wikitext'
data_dir = os.path.join(project_root, 'data', dir_name)
# model_checkpoint = 'gpt2'
model_checkpoint = 'distilgpt2'
# model_checkpoint = 'mjphayes/distilgpt2-finetuned-textbook_dataset'
model_name = model_checkpoint.split('/')[-1]
model_dir = os.path.join(project_root, 'models', model_name)

In [ ]:
try:
    datasets = load_dataset(dataset_name, dataset_variant, cache_dir=data_dir)
except:
    datasets = load_dataset(dataset_name, cache_dir=data_dir)

In [ ]:
datasets

In [ ]:
test_data = datasets['train']

In [ ]:
# def transform(examples):
#     return {"question": examples['question'], "answer": examples['answer']}

In [ ]:
# test_data = test_data.map(transform, num_proc=4, remove_columns=["input", "output", "meta", "id"])

In [ ]:

test_data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, cache_dir=model_dir, use_fast=True, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, cache_dir=model_dir)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def generation(input, model=model):
    prompt = f"Question: {input}? \n Answer:"
    model_inputs = tokenizer([prompt], return_tensors="pt", padding='max_length', max_length=200)
    generated_ids = model.generate(**model_inputs, max_new_tokens=300, do_sample=True)
    bable = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return bable.split('Question: ')[1]

In [ ]:
num = 22
test_answer = generation(test_data[num]['question'])
test_answer

In [ ]:
# mini_val = test_data.select(range(20))
# print(len(mini_val))
# mini_val

In [ ]:
questions = []
ideal_answers = []
gen_answers = []
for q_a in test_data:
    question = q_a['question']
    gen_answer = generation(question).split('Answer: ')[-1]
    ideal = q_a['answer']
    questions.append(question)
    ideal_answers.append(ideal)
    gen_answers.append(gen_answer)



In [ ]:
df = pd.DataFrame({
    'Questions': questions,
    'Ideal': ideal_answers,
    'Generated':gen_answers,
})

In [ ]:
ideal = ideal_answers
generated = gen_answers

In [ ]:
df

In [ ]:
P, R, F1 = score(gen_answers, ideal_answers, lang='en', verbose=True)

In [ ]:
print(f"System level F1 score: {F1.mean():.3f}")

In [ ]:
F1

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(F1, bins=20)
plt.xlabel("score")
plt.ylabel("counts")
plt.show()

In [ ]:
# rag_dataset = load_dataset('domnasrabadi/falcon7bRAG_eval')


In [ ]:
# df = rag_dataset['train'].to_pandas()

In [ ]:
# model_name = 'falcon7b_baseline'

In [ ]:

# df = pd.read_csv('nx_7binstruct_baseline_output2.csv')
df = pd.read_csv('../eval/distilgpt2_eval.csv')
model_name = 'distilgpt2'
# df = pd.read_csv('../eval/distilgpt2-finetuned-textbook_dataset_eval.csv')
# model_name = 'distilgpt2-finetuned'

In [ ]:
# df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
# df.info()

In [ ]:
scorer = BERTScorer(lang="en", rescale_with_baseline=True)

In [ ]:
P, R, F1 = scorer.score(ideal, generated)

In [ ]:
# generated = df['model_answer'].tolist()
# ideal = df['answer'].tolist()

In [ ]:
df['BERTScore_f1'] = F1
df['BERTScore_precision'] = P
df['BERTScore_recall'] = R

In [ ]:
df

In [ ]:
# df.to_csv(os.path.join('../eval', f'falcon7b_rag_eval.csv'), index=False)
df.to_csv(os.path.join('../eval', f'{model_name}_eval.csv'), index=False)

In [ ]:
F1

In [ ]:
print(f"System level F1 score: {F1.mean():.3f}")

In [ ]:
plt.hist(F1, bins=20)
plt.xlabel("score")
plt.ylabel("counts")
plt.suptitle("BERTScore for distilgpt2")
plt.savefig('../eval/bert_score_distilgpt2.png')
plt.show()


In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="TheBloke/Llama-2-7B-Chat-GPTQ")